In [ ]:
cd("../../../")

using Pkg

Pkg.activate(".")
# Pkg.update("StLifeIns")
Pkg.instantiate()
Pkg.precompile()

  Activating project at `C:\Users\User-PC\PythonProjects\Tricycle\TMP`

In [1]:
using StLifeIns
setCPU()
using Distributions

(cashflows)=
# Cashflows

Cashflows are effectively defined by a cashflow structure (amount and timing) paired with a chosen [Contingency](contingency).

```{important}
When specifying cashflow amounts, it's important to specify whether the cashflow is positive or negative, e.g. Premiums are positive, where as Benefits and Expenses are negative.
```

(simple_cf)=
## SimpleCashflows

SimpleCashflows are structures with the primary goal of simplifying the interface for the user. They are used for many policy products and provide predictable and intuitive outputs.

In the background of the code, SimpleCashflows are converted into more mathematically accessible cashflows in the form of [CompleteCashflows](complete_cf).

The scope of SimpleCashflows is limited at this point in time, however, it would be fairly simple to define additional SimpleCashflows if and when needed.

(recurring_cf)=
### RecurringCashflow

A RecurringCashflow is a cashflow that happens at predictable, equally-spaced intervals, e.g. every month, and where the payment value stays the same or grows with a prespecified fixed growth rate.

In [5]:
cf_name = "Premium"
frequency = 12 # monthly
monthly_payment = 100
esc = 0 # (yearly effective growth rate)
first_payment_month = 1
arrears = false # in advance (start of month)
contingency = InForce()

RecurringCashflow(cf_name, monthly_payment, esc, first_payment_month, frequency, arrears, contingency)

RecurringCashflow("Premium", 100.0, 0.0, 1, 12, false, InForce())

```{danger}
A RecurringCashflow should only be used with the [InForce](inforce) contingency.
```

### AnyTimeCashflow

An AnyTimeCashflow is meant for cashflows where the timing of payment is unknown, i.e. it can happen at any time.

In [17]:
cf_name = "Death Benefit"
benefit_amount = -1000000
esc = 0.05 # grows by 5% every year
arrears = true
contingency = OnDeath()

AnyTimeCashflow(cf_name, benefit_amount, esc, arrears, contingency)

AnyTimeCashflow("Death Benefit", -1.0e6, 0.05, true, OnDeath())

```{danger}
An AnyTimeCashflow should only be used with the [OnDeath](inforce) or [OnTermination](ontermination) contingencies.
```

(complete_cf)=
## CompleteCashflows

CompleteCashflows are more immediately mathematically tractable than [SimpleCashflows](simple_cf). This does not, however, mean that they are inherently more complex (as will be seen from the [ZeroCashflow](zero_cf) and [PointCashflow](point_cf)).

(point_cf)=
### PointCashflow

A PointCashflow is a cashflow of which the amount and timing are known, but is still contingent on some [Contingency](contingency).

In [16]:
cf_name = "Survival Benefit"
amount = -1000000
time = 120
arrears = true
contingency = InForce()

PointCashflow(cf_name, amount, time, arrears, contingency)

PointCashflow("Survival Benefit", -1.0e6, 120, true, InForce())

(zero_cf)=
### ZeroCashflow

A cashflow with no payment and therefore also no contingency. It is unlikely that such a cashflow would be specified explicitly by the user, but it can be especially useful in the background code.

In [14]:
cf_name = "Term Assurance Benefit Not Paid"
ZeroCashflow(cf_name)

ZeroCashflow("Term Assurance Benefit Not Paid")

(vector_cf)=
### VectorCashflow

A particularly flexible cashflow that takes a vector of monthly payments, where the position in the vector defines the timing of the payment.

In [19]:
cf_name = "Inconsitent Premiums for 6 months"
payment_amounts = [100, 0, 200, 0, 150, 20]
arrears = false # in advance
contingency = InForce()

VectorCashflow(cf_name, payment_amounts, arrears, contingency)

VectorCashflow("Inconsitent Premiums for 6 months", [100.0, 0.0, 200.0, 0.0, 150.0, 20.0], false, InForce())

```{note}
Where a cashflow can more easily be defined, it is recommended to define a [SimpleCashflow](simple_cf) and an accompanying [complete function](complete).
```

```{hint}
By combining a vector with strategically placed zeroes and a suitable [Contingency](contingency), very complex cashflows can be created.
```

(parallel_cf))=
## ParallelCashflows

Up to this point, while the cashflow objects have been uncertain in the sense that they were linked to a contingency, there has been no uncertainty in the actual payment amount.

ParallelCashflows exist for the cases where the cashflow amount itself is uncertain. An additional dimension is added to store cashflow amounts under every simulation.

```{note}
ParallelCashflows were primarily added to allow expense cashflows to vary with inflation, however, they are more flexible than this and can be used in other advanced circumstances.
```

### ParallelPointCashflow

A ParallelPointCashflow is used when only a single payment at a known time will occur, but the payment amount is variable under different simulations. It is effectively a [PointCashflow](point_cf) with unknown payment amount.

In [82]:
cf_name = "Survival Claim Expense with Inflation"
amount = 100
time = 120 # 10 years from contract start
arrears = true
contingency = InForce()

nsims = 2
inflation_distribution = Normal(0.05, 0.02)
inflation = rand(inflation_distribution, nsims, time)
inflation_fact = (1 .+ inflation).^(1/12)
cum_infl = cumprod(inflation_fact, dims=2)[:, time]
amount = amount * cum_infl

ParallelPointCashflow(cf_name, amount, time, arrears, contingency)

ParallelPointCashflow("Survival Claim Expense with Inflation", [158.94533707693844, 161.93836313165642], 120, true, InForce())

### ParallelVectorCashflow

A ParallelVectorCashflow is used when several payments may be made at potentially unknown times and of unknown amount. A ParallelVectorCashflow is effectively a [VectorCashflow](vector_cf) with unknown payment values.

In [83]:
cf_name = "Regular Expenses with Inflation"
amount = 100
time = 3 # 3 months from contract start
arrears = true
contingency = InForce()

nsims = 2
inflation_distribution = Normal(0.05, 0.02)
inflation = rand(inflation_distribution, nsims, time)
inflation_fact = (1 .+ inflation).^(1/12)
cum_infl = cumprod(inflation_fact, dims=2)
amount = amount * cum_infl

ParallelVectorCashflow(cf_name, amount, arrears, contingency)

ParallelVectorCashflow("Regular Expenses with Inflation", [100.46579098087989 101.08879384989325 101.33123448700545; 100.23416499024376 100.64486523545635 101.26421720958159], true, InForce())